In [1]:
######################################
########## GET data from DB ##########
######################################

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from urllib import parse
from datetime import datetime
from tensorflow.keras.models import load_model

# get connection information from .env file
with open('.env', 'r') as f:
  conn_info = dict(
    tuple(line.strip().split('='))
    for line in f
  )
# create engine for mysql
engine = create_engine((f"mysql://{conn_info['USER']}:{parse.quote(conn_info['PASSWORD'])}@{conn_info['HOST']}:{conn_info['PORT']}/{conn_info['DATABASE']}"))

2024-05-28 15:48:22.036613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = {
  'gugun': '강남구',
  'road_name': '개포로',
  'road_number': '307'
}
sql = f"select * from final_data where gugun='{data['gugun']}' and road_name='{data['road_name']}' and road_num='{data['road_number']}'"

df = pd.read_sql(sql, engine)

In [3]:
size_list = df['size'].unique()
# from today to 6 months after (per month) [today+1month, today+2months, ...]
today = datetime.today()
deal_month_list = [(today + pd.DateOffset(months=i) - datetime(1970, 1, 1)).total_seconds() for i in range(7)][1:]

target_column = ['avg_sale_price']


# preprocessing
df['average_age'] = df['average_age'].replace({'-': np.nan})
df['average_age'] = df['average_age'].astype(float)
df['suicide_rate'] = df['suicide_rate'].replace({'-': np.nan})
df['suicide_rate'] = df['suicide_rate'].astype(float)

# get labelencoder object
import json

with open('./seoul_real_estate/encoding_table_gugun.json', 'r') as f:
  encoding_table_gugun = json.load(f)
with open('./seoul_real_estate/encoding_table_dong.json', 'r') as f:
  encoding_table_dong = json.load(f)
with open('./seoul_real_estate/encoding_table_road.json', 'r') as f:
  encoding_table_road = json.load(f)

# do label encoding
df['gugun'] = df['gugun'].map(encoding_table_gugun)
df['dong'] = df['dong'].map(encoding_table_dong)
df['road_name'] = df['road_name'].map(encoding_table_road)

# drop columns
drop_columns = ['main_bunge', 'sub_bunge', 'danji_name', 'road_num', 'build_year', 'lo', 'la', 'elementary_school_teachers', 'elementary_school_students', 'kindergarten_teachers', 'kindergarten_students']
df.drop(drop_columns, axis=1, inplace=True)

# get sub_data from df
df_sorted = df.sort_values('deal_month', ascending=False)
first_row = df_sorted.iloc[0]
first_row.drop(['deal_month', 'size', 'avg_sale_price'], inplace=True)

# make dataframe for prediction
prediction_df = pd.concat([first_row]*6, axis=1).T
prediction_df['deal_month'] = deal_month_list
prediction_df['size'] = size_list[0]
prediction_df = prediction_df[['gugun', 'dong', 'road_name', 'deal_month', 'size', 'avg_rent_deposit_price', 'hshld_cost', 'elvtr_cnt', 'totprk_ecct', 'cctv_cnt', 'resident_population', 'net_migration', 'average_age', 'foreigner_population', 'stress_cognition_rate', 'subjective_health_level_cognition_rate', 'high_risk_drinking_rate', 'health_life_practice_rate', 'suicide_rate', 'building_age', 'elvtr_ratio', 'prk_ratio', 'cctv_ratio', 'foreigner_ratio', 'student_teacher_ratio']]

/tmp/ipykernel_15591/2205876557.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row.drop(['deal_month', 'size', 'avg_sale_price'], inplace=True)


In [4]:
prediction_df['deal_month'] = deal_month_list
prediction_df['size'] = size_list[0]
display(prediction_df.columns)
prediction_df = prediction_df[['gugun', 'dong', 'road_name', 'deal_month', 'size', 'avg_rent_deposit_price', 'hshld_cost', 'elvtr_cnt', 'totprk_ecct', 'cctv_cnt', 'resident_population', 'net_migration', 'average_age', 'foreigner_population', 'stress_cognition_rate', 'subjective_health_level_cognition_rate', 'high_risk_drinking_rate', 'health_life_practice_rate', 'suicide_rate', 'building_age', 'elvtr_ratio', 'prk_ratio', 'cctv_ratio', 'foreigner_ratio', 'student_teacher_ratio']]

Index(['gugun', 'dong', 'road_name', 'avg_rent_deposit_price', 'hshld_cost',
       'elvtr_cnt', 'totprk_ecct', 'cctv_cnt', 'resident_population',
       'net_migration', 'average_age', 'foreigner_population',
       'stress_cognition_rate', 'subjective_health_level_cognition_rate',
       'high_risk_drinking_rate', 'health_life_practice_rate', 'suicide_rate',
       'building_age', 'elvtr_ratio', 'prk_ratio', 'cctv_ratio',
       'foreigner_ratio', 'student_teacher_ratio', 'deal_month', 'size'],
      dtype='object')

In [5]:
display(prediction_df)

,gugun,dong,road_name,deal_month,size,avg_rent_deposit_price,hshld_cost,elvtr_cnt,totprk_ecct,cctv_cnt,...,subjective_health_level_cognition_rate,high_risk_drinking_rate,health_life_practice_rate,suicide_rate,building_age,elvtr_ratio,prk_ratio,cctv_ratio,foreigner_ratio,student_teacher_ratio
243,0,6,33,1.719590e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758
243,0,6,33,1.722182e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758
243,0,6,33,1.724860e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758
243,0,6,33,1.727539e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758
243,0,6,33,1.730131e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758
243,0,6,33,1.732809e+09,104.43,102833.333333,405.0,11.0,419.0,64.0,...,60.2,12.6,49.4,24.3,38,2.716049,103.45679,15.802469,0.945186,1434.137758


In [6]:
# get prediction

# get model
model = load_model('./seoul_real_estate/trained_model.h5')

# predict
prediction = model.predict(prediction_df)

2024-05-28 15:48:47.816757: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


: 

In [ ]:
display(prediction)

['avg_sale_price']
